In [1]:
import os
os.chdir("../")

# Data Cleaning

This notebook deals with everything associated with data cleaning. Which includes filling in missing values, handling noise, tackling inconsistancies, etc.

## Loading Dataset

In [2]:
import pandas as pd
import plotly.express as px

In [3]:
df = pd.read_csv("data/Asteroid_Updated.csv", low_memory=False)
print(f"Number of (rows, columns) = {df.shape}")

Number of (rows, columns) = (839714, 31)


In [4]:
df.sample(3)

,name,a,e,i,om,w,q,ad,per_y,data_arc,...,UB,IR,spec_B,spec_T,G,moid,class,n,per,ma
258229,NaN,3.051257,0.072386,9.104585,219.281976,207.423067,2.830388,3.272125,5.329989,7191.0,...,NaN,NaN,NaN,NaN,NaN,1.84647,MBA,0.184921,1946.778664,62.292407
155216,NaN,2.668516,0.054510,2.000629,326.909127,46.035094,2.523056,2.813976,4.359262,6671.0,...,NaN,NaN,NaN,NaN,NaN,1.52237,MBA,0.226099,1592.220399,83.644396
751061,NaN,2.839077,0.198508,14.445930,119.394940,218.012150,2.275498,3.402655,4.783810,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,MBA,0.206034,1747.286719,208.217490


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 839714 entries, 0 to 839713
Data columns (total 31 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   name            21967 non-null   object 
 1   a               839712 non-null  float64
 2   e               839714 non-null  float64
 3   i               839714 non-null  float64
 4   om              839714 non-null  float64
 5   w               839714 non-null  float64
 6   q               839714 non-null  float64
 7   ad              839708 non-null  float64
 8   per_y           839713 non-null  float64
 9   data_arc        824240 non-null  float64
 10  condition_code  838847 non-null  object 
 11  n_obs_used      839714 non-null  int64  
 12  H               837025 non-null  float64
 13  neo             839708 non-null  object 
 14  pha             823272 non-null  object 
 15  diameter        137636 non-null  object 
 16  extent          18 non-null      object 
 17  albedo    

## Missing Values

This section deals with handling missing values.

### Identify Missing Columns

In this subsection, I'll identify which columns have missing values. What percentage of the values are missing. I visualize the missing statistics in a bar plot. I, then, chart a course on how to handle the different levels of missing values.

In [6]:
missing = pd.DataFrame(
    df.apply(lambda x: x.isna(), axis=1).sum().sort_values(ascending=True)
).reset_index()

missing.rename(columns={0: "Missing", "index": "Column"}, inplace=True)
missing["Percent"] = missing["Missing"] / df.shape[0] * 100

In [7]:
fig = px.bar(missing[missing.Missing > 0], x="Column", y="Percent", text="Missing")
fig.update_layout(
    height=600,
    width=800,
    title_x=0.5,
    title_text=f"Bar Chart<br><sup>Missing Values of each column</sup>"
)
fig.show()

**Observation 1**

Nearly all values in `rot_per` to `IR` are missing. Predicting them from the existing ones will be hard as there isn't enough data. 

    The best way to deal with these columns is to drop them. If I learn of a better way to handle these missing values, I'll come and deal with them later on.

In [8]:
missing[missing.Percent > 90]

,Column,Missing,Percent
21,name,817747,97.383990
22,rot_per,820918,97.761619
23,spec_B,838048,99.801599
24,BV,838693,99.878411
25,spec_T,838734,99.883294
26,UB,838735,99.883413
27,G,839595,99.985829
28,extent,839696,99.997856
29,GM,839700,99.998333
30,IR,839713,99.999881


**Observation 2**

A big chunk of `diameter` and `albedo` values are missing. 

    Predicting them with a Machine Learning model should be possible from the 20\% data that is available. I'll use a simple deep learning model to do this.

In [9]:
missing[missing.Column.isin(["diameter", "albedo"])]

,Column,Missing,Percent
19,diameter,702078,83.609181
20,albedo,703305,83.755302


**Observation 3**

Some columns have absolutely no missing values. 

    Nothing needs to be done for these columns. I'll use these to help me in imputing other missing values.

In [10]:
missing[missing.Missing == 0]

,Column,Missing,Percent
0,e,0,0.0
1,i,0,0.0
2,om,0,0.0
3,w,0,0.0
4,q,0,0.0
5,class,0,0.0
6,n_obs_used,0,0.0


**Observation 4**

Most columns have $<5\%$ data is missing. 

    These can be filled in using imputation techniques. For numerical columns, I'll use imputation by group median. For categorical, I'll impute by group mode.

In [11]:
missing[(missing.Percent < 5) & (missing.Missing > 0)]

,Column,Missing,Percent
7,per_y,1,0.000119
8,a,2,0.000238
9,n,2,0.000238
10,ad,6,0.000715
11,neo,6,0.000715
12,per,6,0.000715
13,ma,8,0.000953
14,condition_code,867,0.103249
15,H,2689,0.320228
16,data_arc,15474,1.842770


### Dropping Columns

In this subsection, I drop the columns that have more than 90\% of their values missing.

In [12]:
df.drop(
    columns=[
        "name",
        "rot_per",
        "spec_B",
        "spec_T",
        "G",
        "BV",
        "UB",
        "IR",
        "GM",
        "extent",
    ],
    inplace=True,
)

print(f"After dropping, dataframe shape = {df.shape}")

After dropping, dataframe shape = (839714, 21)


### Imputation by Group

In this subsection, I'll impute missing values for columns with less than 5\% of their data missing.

#### Columns with very few missing values

I think grouping by one category will be sufficient to fill in their missing values.

In [13]:
impute_columns = ["per_y", "a", "n", "ad", "per", "ma"]

In [14]:
df[impute_columns] = (
    df.groupby(by="class")
    [impute_columns].apply(lambda x: x.fillna(x.median()))
    .reset_index()[impute_columns]
)

print("After imputing")
df[impute_columns].isna().sum()

After imputing


per_y    0
a        2
n        2
ad       6
per      6
ma       2
dtype: int64

Seems like some values couldn't be imputed in this case. This is due to all values of a particular group being missing. I'll impute these by grouping with some other column.

##### `neo` column

In [18]:
df[df.neo.isna()]

,a,e,i,om,w,q,ad,per_y,data_arc,condition_code,...,H,neo,pha,diameter,albedo,moid,class,n,per,ma
824768,3.225987,1.201134,122.741706,24.596910,241.810536,0.255912,3.346375,5.794315,80.0,NaN,...,22.0800,NaN,N,NaN,NaN,0.095821,HYA,0.170102,2116.373688,120.046605
824906,3.974994,1.000826,142.636401,276.230222,326.001880,6.417550,4.596751,7.925248,798.0,NaN,...,10.6440,NaN,N,NaN,NaN,5.464480,HYA,0.124365,2894.696852,234.401961
836687,39.377318,1.000056,138.380218,130.468071,287.870052,5.858483,43.797854,247.102654,201.0,NaN,...,13.4340,NaN,N,NaN,NaN,5.116310,HYA,0.003989,90254.244354,156.398311
838330,47.952767,1.000000,89.826795,300.387873,59.801951,8.828361,56.797913,332.069285,111.0,NaN,...,9.5996,NaN,N,NaN,NaN,8.276200,PAA,0.002968,121288.306433,75.896315
838493,30.356806,1.000000,155.720291,42.337498,55.893065,4.997530,34.044743,167.260082,67.0,NaN,...,12.7600,NaN,N,NaN,NaN,4.063860,PAA,0.005893,61091.744882,283.309071
839354,69.118531,1.001016,113.525939,2.699366,181.618474,3.618257,101.406892,574.645422,18.0,NaN,...,10.4200,NaN,N,NaN,NaN,2.621050,HYA,0.001715,209889.240382,3.562422


I can use `pha` column to group and impute. Other categorical columns, 

* `condition_code` is null for all these rows.
* `class` can't be used. Because all instances of the groups have null `neo` values.

In [22]:
df.groupby("pha").neo.apply(lambda x: x.mode().iloc[0])

pha
N    N
Y    Y
Name: neo, dtype: object

Rows with `pha` of **N** has a `neo` mode of **N**. So, I'll impute the missing `neo` values with this.

In [23]:
df.neo.fillna("N", inplace=True)

To confirm, I should have zero missing values now.

In [24]:
df.neo.isna().sum()

0

##### `condition_code` column

In [27]:
df["condition_code"] = (
    df.groupby(["class", "neo"])
    .transform(lambda x: x.fillna(x.mode().iloc[0]))
    .condition_code
)

In [28]:
df[df.condition_code.isna()]

,a,e,i,om,w,q,ad,per_y,data_arc,condition_code,...,H,neo,pha,diameter,albedo,moid,class,n,per,ma
824768,3.225987,1.201134,122.741706,24.596910,241.810536,0.255912,3.346375,5.794315,80.0,NaN,...,22.0800,N,N,NaN,NaN,0.095821,HYA,0.170102,2116.373688,120.046605
824906,3.974994,1.000826,142.636401,276.230222,326.001880,6.417550,4.596751,7.925248,798.0,NaN,...,10.6440,N,N,NaN,NaN,5.464480,HYA,0.124365,2894.696852,234.401961
836687,39.377318,1.000056,138.380218,130.468071,287.870052,5.858483,43.797854,247.102654,201.0,NaN,...,13.4340,N,N,NaN,NaN,5.116310,HYA,0.003989,90254.244354,156.398311
838330,47.952767,1.000000,89.826795,300.387873,59.801951,8.828361,56.797913,332.069285,111.0,NaN,...,9.5996,N,N,NaN,NaN,8.276200,PAA,0.002968,121288.306433,75.896315
838493,30.356806,1.000000,155.720291,42.337498,55.893065,4.997530,34.044743,167.260082,67.0,NaN,...,12.7600,N,N,NaN,NaN,4.063860,PAA,0.005893,61091.744882,283.309071
839354,69.118531,1.001016,113.525939,2.699366,181.618474,3.618257,101.406892,574.645422,18.0,NaN,...,10.4200,N,N,NaN,NaN,2.621050,HYA,0.001715,209889.240382,3.562422


In [30]:
df.groupby("neo").apply(lambda x: x.mode().iloc[0]).condition_code

neo
N    0
Y    0
Name: condition_code, dtype: object

Rows with `neo` value **N** has mostly have a `condition_code` value of 0.

In [31]:
df.condition_code.fillna(0, inplace=True)

In [32]:
df.condition_code.isna().sum()

0

##### `pha` column

In [39]:
df["pha"] = (
    df.groupby(["neo", "class"])
    .transform(lambda x: x.fillna(x.mode().iloc[0]))
    .pha
)

df.pha.isna().sum()

0